In [8]:
# Developed by Samuel Horovatin at the University of Saskatchewan
# July, 2021
from plantcv import plantcv as pcv
import matplotlib
import os

In [22]:
matplotlib.rcParams["figure.figsize"] = [8, 8]
class options:
    def __init__(self):
        # Input image path/filename
        self.image_dir = "/student/sch923/Thesis/data/lentil/"
        # Debug mode = None, "plot", or "print"
        self.debug = None
        # Store output images (True/False)
        self.writeimg = False
        # Results path/filename
        self.result = "results.txt"
        # Image output directory path
        self.outdir = "/student/sch923/Thesis/Lentil/Processed/"

# Initialize options
args = options()
# Set PlantCV debug mode to input debug method
pcv.params.debug = args.debug
# Increase text size in plots
pcv.params.text_size = 20
pcv.params.text_thickness = 10

In [26]:
for lentil_image in os.listdir(args.image_dir):
    img, path, filename = pcv.readimage(filename=args.image_dir+lentil_image)
    img = img[:, :1400, :] #removes mirror
    cs_plot = pcv.visualize.colorspaces(rgb_img=img, original_img=False)
    s = pcv.rgb2gray_hsv(rgb_img=img, channel='s')
    s_thresh = pcv.threshold.binary(gray_img=s, threshold=60, max_value=255, object_type='light') # threshold is 60 to remove little lentil flecks
    b = pcv.rgb2gray_lab(rgb_img=img, channel='b')
    b_thresh = pcv.threshold.binary(gray_img=b, threshold=137, max_value=255, object_type='light')
    a = pcv.rgb2gray_lab(rgb_img=img, channel='a')
    a_thresh = pcv.threshold.binary(gray_img=a, threshold=133, max_value=255, object_type='light')
    bs = pcv.logical_or(bin_img1=s_thresh, bin_img2=b_thresh)
    bsa = pcv.logical_or(bin_img1=bs, bin_img2=a_thresh)
    bsa_fill1 = pcv.fill(bin_img=bsa, size=200) # Fill small noise
    bsa_fill2 = pcv.dilate(gray_img=bsa_fill1, ksize=6, i=2)
    # bsa_fill3 and bsa_fill4 are to get rid of tiny lentil flakes
    bsa_fill3 = pcv.erode(bsa_fill2, ksize=10, i=4)
    bsa_fill4 = pcv.dilate(gray_img=bsa_fill3, ksize=20, i=4)
    filled_mask1 = pcv.fill_holes(bsa_fill4)
    id_objects, obj_hierarchy = pcv.find_objects(img=img, mask=filled_mask1)
    obj, mask = pcv.object_composition(img=img, contours=id_objects, hierarchy=obj_hierarchy)
    crop_img = pcv.auto_crop(img=img, obj=obj, padding_x=5, padding_y=5, color='black')
    pcv.print_image(crop_img, args.outdir+"cropped_"+lentil_image)

In [24]:
img, path, filename = pcv.readimage(filename="/student/sch923/Thesis/data/lentil/17085814-44.png")
img = img[:, :1400, :] #removes mirror
cs_plot = pcv.visualize.colorspaces(rgb_img=img, original_img=False)
s = pcv.rgb2gray_hsv(rgb_img=img, channel='s')
s_thresh = pcv.threshold.binary(gray_img=s, threshold=60, max_value=255, object_type='light') # threshold is 60 to remove little lentil flecks
b = pcv.rgb2gray_lab(rgb_img=img, channel='b')
b_thresh = pcv.threshold.binary(gray_img=b, threshold=137, max_value=255, object_type='light')
a = pcv.rgb2gray_lab(rgb_img=img, channel='a')
a_thresh = pcv.threshold.binary(gray_img=a, threshold=133, max_value=255, object_type='light')
bs = pcv.logical_or(bin_img1=s_thresh, bin_img2=b_thresh)
bsa = pcv.logical_or(bin_img1=bs, bin_img2=a_thresh)
bsa_fill1 = pcv.fill(bin_img=bsa, size=200) # Fill small noise
bsa_fill2 = pcv.dilate(gray_img=bsa_fill1, ksize=6, i=2)
# bsa_fill3 and bsa_fill4 are to get rid of tiny lentil flakes
bsa_fill3 = pcv.erode(bsa_fill2, ksize=10, i=4)
bsa_fill4 = pcv.dilate(gray_img=bsa_fill3, ksize=10, i=4)
filled_mask1 = pcv.fill_holes(bsa_fill4)
id_objects, obj_hierarchy = pcv.find_objects(img=img, mask=filled_mask1)
obj, mask = pcv.object_composition(img=img, contours=id_objects, hierarchy=obj_hierarchy)
crop_img = pcv.auto_crop(img=img, obj=obj, padding_x=5, padding_y=5, color='black')
pcv.print_image(crop_img, args.outdir+"cropped_"+lentil_image)